In [1]:
!pip install -U langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0

In [3]:
import inspect

from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [4]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [5]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [6]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [7]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [9]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi!'),
  AIMessage(content="what's up?")]}

In [10]:
import os
os.environ["OPENAI_API_KEY"] = "your api key"

In [11]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [12]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:


New human question: hi
Response:

> Finished chain.


{'question': 'hi',
 'chat_history': '',
 'text': ' Hello! How are you doing today?'}

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [14]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Hello! How can I assist you today?'}

In [15]:
from langchain.chains.conversation.memory import ConversationBufferMemory
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [16]:
conversation = ConversationChain(
    llm=llm,
)

In [17]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

In [18]:
print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

In [19]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [20]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning! How can I assist you today?'}

In [21]:
conversation_buf("Can you please tell me your name AI?")

{'input': 'Can you please tell me your name AI?',
 'history': 'Human: Good morning AI!\nAI: Good morning! How can I assist you today?',
 'response': 'Of course! My name is OpenAI. How can I assist you today?'}

In [22]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [23]:
count_tokens(
    conversation_buf,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)


Spent a total of 271 tokens


'That sounds like a fascinating concept! The idea of a Big Hairy Audacious Goal (BHAG) was first introduced by James Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." They define a BHAG as a long-term goal that is both ambitious and inspiring, capable of energizing and aligning the efforts of employees and stakeholders. The purpose of a BHAG is to push an organization beyond its current capabilities and create a sense of purpose and direction. It\'s a powerful tool for motivating and guiding the actions of individuals and teams towards a common objective.'

In [24]:
count_tokens(
    conversation_buf,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)

Spent a total of 483 tokens


'Absolutely! The BHAG can indeed help an organization focus on a long-term goal by providing a clear and compelling vision for the future. It serves as a guiding star that helps leaders make strategic decisions and align resources towards achieving that goal. By setting a BHAG, organizations create a sense of urgency as they understand the importance of taking action and making progress towards their ambitious goal. This sense of urgency can drive innovation, collaboration, and a relentless pursuit of excellence. Moreover, a BHAG can attract and retain top talent by offering a compelling vision and a sense of purpose. It acts as a magnet for individuals who are motivated by challenging and meaningful work. By embracing their BHAG, organizations can galvanize their employees and stakeholders, fostering a strong and unified culture where everyone is motivated and aligned towards achieving success.'

In [25]:
count_tokens(
    conversation_buf,
    "What concept this book introduces?"
)

Spent a total of 510 tokens


'This book introduces the concept of a Big Hairy Audacious Goal (BHAG).'

In [26]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI: Good morning! How can I assist you today?
Human: Can you please tell me your name AI?
AI: Of course! My name is OpenAI. How can I assist you today?
Human: This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,     and inspiring goal that can motivate and align the efforts of employees and stakeholders.
AI: That sounds like a fascinating concept! The idea of a Big Hairy Audacious Goal (BHAG) was first introduced by James Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." They define a BHAG as a long-term goal that is both ambitious and inspiring, capable of energizing and aligning the efforts of employees and stakeholders. The purpose of a BHAG is to push an organization beyond its current capabilities and create a sense of purpose and direction. It's a powerful tool for motivating and guiding the actions of individuals and teams towards a common objective.
Human: The BHAG 

In [27]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI: Good morning! How can I assist you today?
Human: Can you please tell me your name AI?
AI: Of course! My name is OpenAI. How can I assist you today?
Human: This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,     and inspiring goal that can motivate and align the efforts of employees and stakeholders.
AI: That sounds like a fascinating concept! The idea of a Big Hairy Audacious Goal (BHAG) was first introduced by James Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." They define a BHAG as a long-term goal that is both ambitious and inspiring, capable of energizing and aligning the efforts of employees and stakeholders. The purpose of a BHAG is to push an organization beyond its current capabilities and create a sense of purpose and direction. It's a powerful tool for motivating and guiding the actions of individuals and teams towards a common objective.
Human: The BHAG 

In [28]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)


In [29]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [30]:
# without count_tokens we'd call `conversation_sum("Can you please tell me your name AI?!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum,
    "Can you please tell me your name AI?"
)

Spent a total of 283 tokens


'Of course! My name is OpenAI Assistant. How can I assist you today?'

In [31]:
count_tokens(
    conversation_sum,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)

Spent a total of 797 tokens


'That sounds like an interesting concept! The idea of a Big Hairy Audacious Goal (BHAG) was first introduced by James Collins and Jerry Porras in their book "Built to Last: Successful Habits of Visionary Companies." They describe a BHAG as a clear and compelling goal that is both audacious and visionary. It is a goal that goes beyond what seems possible in the present and requires significant effort and innovation to achieve. BHAGs are intended to inspire and motivate employees and stakeholders, helping to align their efforts towards a common purpose. They can be a powerful tool for driving organizational success and transformation. Is there anything specific you would like to know about BHAGs or how they can be applied in a business context?'

In [32]:
count_tokens(
    conversation_sum,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)


Spent a total of 1260 tokens


'Absolutely! Embracing a BHAG can indeed help an organization focus on a long-term goal and create a sense of urgency. By setting a bold and ambitious goal, companies can rally their employees and stakeholders around a shared vision, which in turn can foster a sense of purpose and motivation within the organization. This can lead to increased productivity, innovation, and overall success.\n\nAdditionally, having a BHAG can help attract and retain top talent. Talented individuals are often drawn to organizations that have a clear and compelling vision, as it provides them with a sense of purpose and the opportunity to work towards something meaningful. A BHAG can serve as a powerful tool for employee recruitment and retention, as it showcases the organization\'s ambition and potential for growth.\n\n"Built to Last" by James Collins and Jerry Porras is indeed a great resource for learning more about BHAGs and how they can be applied in a business context. If you\'d like, I can provide mo

In [33]:
count_tokens(
    conversation_sum,
    "What concept this book introduces?"
)

Spent a total of 933 tokens


"I'm sorry, but I don't have enough information to answer your question. Could you please provide more context or clarify what book you are referring to?"

In [34]:
count_tokens(
    conversation_sum,
    "the book encourages readers of what?"
)

Spent a total of 1013 tokens


'I apologize for the confusion, but could you please provide more information or clarify which book you are referring to?'

In [35]:
print(conversation_sum.memory.buffer)

The human asks the AI for its name, and the AI responds that its name is OpenAI Assistant and asks how it can assist the human today. The human introduces the concept of a Big Hairy Audacious Goal (BHAG) and the AI explains that a BHAG is a clear and compelling goal that goes beyond what seems possible in the present and requires significant effort and innovation to achieve. The AI mentions that BHAGs inspire and motivate employees and stakeholders, helping to align their efforts towards a common purpose. The human adds that a BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The AI agrees, stating that embracing a BHAG can foster a sense of purpose and motivation within the organization, leading to increased productivity, innovation, and overall success. Having a BHAG can also attract and retain top talent, as talented individuals are drawn to organizations with a clear and compelling vision. The AI offers to provide

In [36]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=1)
)

In [37]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

In [38]:
count_tokens(
    conversation_bufw,
    "Good morning AI!"
)

Spent a total of 77 tokens


'Good morning! How can I assist you today?'

In [39]:

count_tokens(
    conversation_bufw,
    "I want to know more about Singularity, what it is and when it is coming."
)


Spent a total of 251 tokens


"Singularity refers to a hypothetical point in time when artificial intelligence surpasses human intelligence and becomes capable of self-improvement, leading to an exponential increase in technological advancements. It is often associated with the concept of superintelligence, where AI systems possess cognitive abilities far beyond those of humans.\n\nAs for when Singularity will occur, it is uncertain and highly debated among experts. Some believe it could happen within the next few decades, while others think it is still far off in the future. The timeline for Singularity depends on various factors, including advancements in AI research, computing power, and the development of new technologies.\n\nIt's important to note that Singularity is a speculative concept, and its exact nature and potential consequences are still subjects of intense discussion and study."

In [40]:

count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 261 tokens


'Sure, please let me know what specific information you would like to know about Singularity.'

In [41]:
count_tokens(
    conversation_bufw,
    "Give me the technology behind it"
)


Spent a total of 371 tokens


'The technology behind Singularity is quite complex and multidisciplinary. One of the key components is artificial intelligence (AI), which involves the development of algorithms and systems that can mimic human intelligence and perform tasks that typically require human intelligence, such as problem-solving, learning, and decision-making.\n\nAnother important technology is machine learning, a subset of AI that focuses on teaching computers to learn from and make predictions or decisions based on data. This involves training algorithms on large datasets and allowing them to automatically improve their performance over time.\n\nFurthermore, Singularity also relies on advancements in fields like robotics, natural language processing, computer vision, and data analytics. Robotics plays a crucial role in creating physical embodiments for AI systems, allowing them to interact with the physical world. Natural language processing enables machines to understand and generate human language, mak

In [42]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 381 tokens


"I'm sorry, but I don't know what you want to know. Could you please provide more specific information or ask a specific question?"

In [43]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

In [44]:
conversation_sum_bufw = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=650
    ),
)


In [45]:
print(conversation_sum_bufw.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
